In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Request and Response Logging with Gemini

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/logging/intro_request_response_logging.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Flogging%2Fintro_request_response_logging.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/logging/intro_request_response_logging.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/logging/intro_request_response_logging.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<p>
<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/logging/intro_request_response_logging.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/logging/intro_request_response_logging.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/logging/intro_request_response_logging.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/logging/intro_request_response_logging.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/logging/intro_request_response_logging.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
</p>

| Authors |
| --- |
| [Eric Dong](https://github.com/gericdong) |

## Overview

Vertex AI allows you to log samples of requests and responses for Gemini models. This capability is useful for analyzing model usage, debugging issues, and understanding how your model is performing. The logs are saved directly to a BigQuery table that you specify, making them easy to query and analyze.

### Objectives

In this tutorial, you'll learn how to:

- Enable logging to an automatically created BigQuery destination.
- Send a request to the model to generate a log entry.
- Programmatically identify the auto-created dataset and table.
- Verify the logged content in BigQuery.
- Disable request-response logging.

## Getting Started

### Install the Vertex AI SDK and other required libraries

> ⚠️ This logging feature is a preview feature and is only available in the `preview` module of the Vertex AI SDK.

In [ ]:
%pip install --upgrade --quiet google-cloud-aiplatform google-cloud-bigquery

### Authenticate your notebook environment

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set your project information

Please update the following variables with your Google Cloud project details and the desired BigQuery destination for your logs.

In [ ]:
import os

# fmt: off
PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
# fmt: on
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1"  # @param {type: "string"}

### Import libraries


In [ ]:
import time
import warnings

import vertexai
from IPython.display import display
from google.api_core import exceptions
from google.cloud import bigquery

# Use the preview module for this feature
from vertexai.preview.generative_models import GenerativeModel

# Ignore all warnings
warnings.filterwarnings("ignore")

### Initialize the Vertex AI SDK for Python


In [ ]:
vertexai.init(project=PROJECT_ID, location=LOCATION)

### Supported Models

Learn more about [supported models for logging](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/request-response-logging#supported_api_methods_for_logging). This tutorial uses the Gemini 2.5 Flash model.

In [ ]:
# fmt: off
MODEL_ID = "gemini-2.5-flash"  # @param {type: "string"}
# fmt: on

model = GenerativeModel(MODEL_ID)

## Enable request-response logging

To enable logging, we'll use the `set_request_response_logging_config` method on a `GenerativeModel` instance. This method can be used to both create a new logging configuration or update an existing one.

You need to provide a few key parameters:

- `enabled`: Set to `True` to turn on logging.
- `sampling_rate`: A value from `0.0` to `1.0` indicating the fraction of requests to log. `1.0` means log 100% of requests.
- `bigquery_destination`: The path to your BigQuery table in the format `bq://PROJECT_ID.DATASET_NAME.TABLE_NAME`.

**Notes**:

- To enable logging with automatic resource creation, we will set the `bigquery_destination` to only specify the project ID. When this method is used:
   - A **new BigQuery dataset** is created with a name following the pattern `ENDPOINT_DISPLAY_NAME_ENDPOINT_ID`.
   - A **new table** is created within that dataset with the name `request_response_logging`.

- If a configuration already exists, the API will return an `AlreadyExists` error. The code below wraps the call in a `try...except` block to handle this case gracefully.

In [ ]:
output_uri = None

try:
    logging_config_result = model.set_request_response_logging_config(
        enabled=True,
        sampling_rate=1.0,
        bigquery_destination=f"bq://{PROJECT_ID}",
    )
    print("Successfully enabled request-response logging.")

    output_uri = logging_config_result.logging_config.bigquery_destination.output_uri
    print(f"Retrieved Output URI: {output_uri}")

except exceptions.AlreadyExists:
    print("Request-response logging is already enabled for this model.")

## Generate Content to Create Logs

After enabling the configuration, we need to send a request to the model. This first request will trigger the creation of the dataset and table in BigQuery, and the request/response will be logged.

> ℹ️ Note: It can take a few minutes for the resources to be created and for the first log to appear.

In [ ]:
response = model.generate_content("Why is the sky blue?")
print(response.text)

### Wait for the BQ dataset and table to be created and populated

In [ ]:
# It can take a few minutes for the dataset to be created and for logs to show up
print("Waiting 2 minutes for resources and logs to propagate...")
time.sleep(120)

## Identify Log Destination

Since the dataset name is generated automatically, we need to discover it before we can query the logs. Since the SDK's `set_request_response_logging_config` method is idempotent (it creates or updates), we can use it to retrieve the current configuration by making a trivial change and then immediately changing it back.

The returned object from the successful update call will contain the full configuration, including the output_uri you want.

In [ ]:
if output_uri is None:
    # Define the original and a temporary sampling rate for the toggle
    original_sampling_rate = 1.0
    temp_sampling_rate = 0.99

    try:
        # Update the config with a temporary value to get the return object.
        print(
            f"Temporarily updating sampling_rate to {temp_sampling_rate} to fetch config..."
        )
        config = model.set_request_response_logging_config(
            enabled=True,
            sampling_rate=temp_sampling_rate,
            bigquery_destination=f"bq://{PROJECT_ID}",
        )

        # Extract the output_uri from the returned object.
        output_uri = config.logging_config.bigquery_destination.output_uri
        print(f"\nSuccessfully retrieved Output URI: {output_uri}")

    finally:
        # Always change the setting back to the original value, even if errors occur.
        if output_uri:  # Only run this if the config was successfully fetched
            print(f"\nRestoring original sampling_rate to {original_sampling_rate}...")
            model.set_request_response_logging_config(
                enabled=True,
                sampling_rate=original_sampling_rate,
                bigquery_destination=f"bq://{PROJECT_ID}",
            )
            print("Configuration restored.")

## Query Logs

Now that you have the `output_uri`, you can use the BigQuery client library to parse it, query the correct table, and display the logs.

### Understanding the BigQuery Table Schema

When you enable logging, Vertex AI writes the logs to a BigQuery table with a predefined schema. Here are the fields you will find in that table:

- **`api_method`** (STRING): The API method used, such as `generateContent` or `streamGenerateContent`.
- **`deployed_model_id`** (STRING): The ID for a tuned model that has been deployed to an endpoint.
- **`endpoint`** (STRING): The resource name of the endpoint if a tuned model is deployed.
- **`full_request`** (JSON): The complete `GenerateContentRequest`.
- **`full_response`** (JSON): The complete `GenerateContentResponse`.
- **`logging_time`** (TIMESTAMP): The time the log was recorded, which is roughly when the response was returned.
- **`metadata`** (JSON): Contains metadata about the call, including request latency.
- **`model`** (STRING): The resource name of the model used.
- **`model_version`** (STRING): The version of the model, which is often `"default"` for Gemini models.
- **`otel_log`** (JSON): Logs formatted in the OpenTelemetry schema, which is only available if `otel_logging` is enabled in the configuration.
- **`request_id`** (NUMERIC): The auto-generated integer ID for the API request.
- **`request_payload`** (STRING): Included for logging with partner models and for backward compatibility.
- **`response_payload`** (STRING): Included for logging with partner models and for backward compatibility.

### ⚠️ Important Limitation

It's important to note that request-response pairs that are larger than the **10MB** row limit for the BigQuery write API will not be recorded in the table.


In [ ]:
# Parse the URI to get project, dataset, and table IDs
uri_parts = output_uri.replace("bq://", "").split(".")
if len(uri_parts) != 3:
    raise ValueError(
        "The output_uri format is incorrect. Expected 'bq://project.dataset.table'"
    )

project_id, dataset_id, table_id = uri_parts

# Construct the full table ID for the query, ensuring it's properly quoted
full_table_id = f"`{project_id}.{dataset_id}.{table_id}`"
print(f"Querying table: {full_table_id}")

# Query the BigQuery table
bq_client = bigquery.Client(project=project_id)

# Construct a SQL query to get the 10 most recent logs
query = f"""
    SELECT
        logging_time,
        full_request,
        full_response,
        model
    FROM {full_table_id}
    ORDER BY logging_time DESC
    LIMIT 10
"""

# Execute the query and load the results into a pandas DataFrame
df = bq_client.query(query).to_dataframe()

if df.empty:
    print(
        "\\nNo logs found in the table. It may take a few minutes for new logs to appear."
    )
else:
    print(f"\\nSuccessfully fetched {len(df)} recent log(s):")
    display(df)

## Disable Request-Response Logging

You can disable logging at any time by calling the same `set_request_response_logging_config` method and setting `enabled=False`.

In [ ]:
model.set_request_response_logging_config(
    enabled=False,
    sampling_rate=1.0,
    bigquery_destination=f"bq://{PROJECT_ID}",
)

print("Request-response logging has been disabled.")

## What's next

You can now navigate to your BigQuery project to view the table and analyze the logged data. To learn more, check out the official documentation on [logging requests and responses](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/request-response-logging).